#### Componentes Fundamentais:

- **Models (Modelos):** LangChain serve como uma interface padrão que permite interações com uma ampla gama de Grandes Modelos de Linguagem (LLMs).

- **Chains (Cadeias):** Como seu nome implica, _cadeias_ são o núcleo dos fluxos de trabalho do LangChain. Combinam LLMs com outros componentes, criando aplicativos por meio da execução de uma sequência de funções.

- **Prompts (Instruções):** Os prompts são as instruções apresentadas a um LLM. Geralmente, a "arte" de redigir prompts que efetivamente entregam o contexto necessário para que o LLM interprete a entrada e a saída da estrutura da maneira mais útil para você é chamada de engenharia de prompt.

- **Indexes (Índices):** Para realizar determinadas tarefas, as LLMs precisarão acessar fontes de dados externas específicas não incluídas em seu conjunto de dados de treinamento, como documentos internos, e-mails ou conjuntos de dados. LangChain refere-se coletivamente a essa documentação externa como “índices"."

- **Memory (Memória):** Por padrão, os LLMs não têm memória de longo prazo de conversas anteriores (a menos que o histórico do chat seja usado como entrada para uma consulta). O LangChain soluciona esse problema com utilitários simples para adicionar memória a um sistema, com opções que vão desde a retenção total de todas as conversas até a retenção de um resumo da conversa até a retenção das _n_ trocas mais recentes.

- **Agents/Tools (Agentes/Ferramentas):** Os agentes do LangChain podem usar um determinado modelo LLM como um "mecanismo de raciocínio" para determinar quais ações tomar. Ao criar uma cadeia para um agente, as entradas contêm:

	- uma lista de ferramentas disponíveis para serem aproveitadas.
	- entrada do usuário (como prompts e consultas).
	- quaisquer etapas relevantes executadas anteriormente.

In [ ]:
%run ../helpers/00-llm.ipynb

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from helpers.llm import initialize_llm, logger, pretty_print

llm, _, _ = initialize_llm()

logger.info("LLM and embeddings initialized successfully.")


In [ ]:
### O ChatModel é um componente LangChain então ele possui o protocolo invoke()

In [ ]:
resposta = llm.invoke("Olá como você está e o que você é capaz de fazer?")
     
pretty_print(resposta)  

#### Criando a conversa. 

Lembrando que os ChatModels recebem como entrada uma lista de mensagem. Assim o LangChain automaticamente converte isso na estrutura que o modelo LLM precisa receber para responder.

In [ ]:

# Forma 1 de escrever:
mensagens = [
			 SystemMessage(content="Você é um especialista em astrofísica."),
			 HumanMessage(content="Qual a distancia do sol até a terra?"),
			 AIMessage(content="O Sol está a 49.600.000 km de distância da Terra."),
			 HumanMessage(content="E a distância da terra até marte?"),
]

# Forma 2 de escrever:
# mensagens = [
# 			 ("system", "Você é um especialista em astrofísica."),
# 			 ("user", "Qual a distancia do sol até a terra?"),
# 			 ("assistant", "O Sol está a&nbsp;49.600.000 km de distância da Terra."),
#            ("user", "E a distância da terra até marte?"),
# ]

# Como a entrada do usuário é a ultima mensagem da lista, você pode dá invoke usando a lista de pensagens contendo o histórico de conversação.
resposta = llm.invoke(mensagens)

pretty_print(resposta)
 

print("-------------------------------------------------------------------")

Vamos agora criar um chat, ou seja, vamos criar uma lista que vai crescendo dinamicamente com a entrada do usuário simulando uma conversa com ChatGPT.

Vamos agora simular o streaming de dados dos modelos (quando compatível) onde cada token é gerado em tempo de execução. Vamos repetir o código anterior e ao invés de invoke, vamos chamar o modelo usando a função assíncrona astream: